# Fermi Tool in Python

## This notebook is for the use of Fermi Tools.

To be able to run any enviornment in the Jupyter Notebook you must run
>`conda install nb_conda`

continue through installation and restart Jupyter Notebook.

When opening with new kernel it will prompt what enviornment to work with and you will see
>`conda env:fermi`

As with any enviornment you might need to reinstall packages


When using Fermi please refer to Analysis Threads [here](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/)
for any other issues

# September 10, 2017 Flare
    15:30 - 05:00
    X8.2 GOES
    
The flare has its peak gamma ray emission at around 15:45-16:00  
There is SGRE (sustained gamma ray emission) until approx 05:00

# Import all packages and modules

In [3]:
import time
import math
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from astropy.io import fits 
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import solar_system_ephemeris, get_body 
from gt_apps import filter, evtbin, maketime, expCube, expMap, diffResps, like, TsMap
from coordinates import equatorial

# **Set up all variables**

You can also find the correct evclass, evtype and IRF [here](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data_Exploration/Data_preparation.html) here for the likilihood analysis from the Fermi Analysis Threads

In [4]:
#files
flaredate = 'september102017'
flarefile = 'september102017,1530-0500'
header_file = flarefile + '_PH00.fits'
sc_file = flarefile + '_SC00.fits'
data_file = flarefile + '_events.list'

### **We run this one for solar flares IMPULSIVE PHASE**
---
Must make following change in GtSelect
>`filter['tmax'] = time_start_MET + 5400`

In [5]:
"""change the infile on gtselect to ['@' + data_file]"""

evclass = 65536
evtype = 3
irf_name = 'iso_P8R3_TRANSIENT015S_V2_v1'
irfs = 'P8R3_TRANSIENT015S_V2'
filter_exp = '(DATA_QUAL>0||DATA_QUAL==-1)&&(LAT_CONFIG==1)'
optimizer = 'NewMinuit'
roicut = 'yes'

### **We run this one for solar flares EXTENDED EMISSION (point sources)**
---
Must make following change in GtSelect
>`filter['tmin'] = time_start_MET + 5400`

In [ ]:
evclass = 128
evtype = 3
irf_name = 'iso_P8R3_SOURCE_V2_v1'
irfs = 'P8R3_SOURCE_V2'
filter_exp = '(DATA_QUAL>0)&&(LAT_CONFIG==1)'
optimizer = 'NewMinuit'
roicut = 'no'

## **Select Parameters**

In [6]:
#get parameters from Fermi photon event file
hdulist = fits.open(header_file)
hdu = hdulist[1]

roi_data = (hdu.header['DSVAL3']).split(',')
roi = int(float(((roi_data[2])[:-1]))) #degrees
time_start = Time(hdu.header['DATE-OBS'], format='fits')
time_end = Time(hdu.header['DATE-END'], format='fits') 
time_step = 0.5 #in units of days
time_start_MET = hdu.header['TSTART'] #MET (s)
time_end_MET = hdu.header['TSTOP'] #MET (s)


dec = float((roi_data[1])) #degrees
ra = float(((roi_data[0])[7:])) #degrees
energyrange = (hdu.header['DSVAL5']).split(':')
emin = float(energyrange[0]) #MeV
emax = float(energyrange[1]) #MeV
num_bins = int(math.log10(emax/emin)*10)
zmax = 100 #deg
bin_size = 0.2 #deg/pix
dim = int((2*roi)/bin_size) #gives dimension of cmap diameter/bin size = num pixels

hdulist.close()

## **GtSelect**

In [7]:
filtered_file = flaredate + '_flare.fits'
filter['evclass'] = evclass
filter['evtype'] = evtype
filter['ra'] = ra
filter['dec'] = dec
filter['rad'] = roi
filter['emin'] = emin
filter['emax'] = emax
filter['zmax'] = zmax
filter['tmin'] = time_start_MET
filter['tmax'] = time_start_MET + 5400
filter['infile'] = '@' + data_file
filter['outfile'] = filtered_file
filter.run()

time -p gtselect infile=@september102017,1530-0500_events.list outfile=september102017_flare.fits ra=168.86425 dec=4.786 rad=10.0 tmin=526750205.0 tmax=526755605.0 emin=100.0 emax=500000.0 zmin=0.0 zmax=100.0 evclass=65536 evtype=3 convtype=-1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"


Done.
real 1.90
user 1.33
sys 0.15


## **GtMaketime**

In [8]:
gti_file = flaredate + '_flare_gti.fits'
maketime['scfile'] = sc_file
maketime['filter'] = filter_exp
maketime['roicut'] = roicut
maketime['evfile'] = filtered_file
maketime['outfile'] = gti_file
maketime.run()

time -p gtmktime scfile=september102017,1530-0500_SC00.fits sctable="SC_DATA" filter="(DATA_QUAL>0||DATA_QUAL==-1)&&(LAT_CONFIG==1)" roicut=yes evfile=september102017_flare.fits evtable="EVENTS" outfile="september102017_flare_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"


real 0.44
user 0.32
sys 0.11


## **GtExposureCube**

In [9]:
ltcube_file = flaredate + '_flare_lt.fits'
expCube['evfile'] = gti_file
expCube['scfile'] = sc_file
expCube['outfile'] = ltcube_file
expCube['zmax'] = zmax
expCube['dcostheta'] = 0.025
expCube['binsz'] = 1
expCube.run() 

time -p gtltcube evfile="september102017_flare_gti.fits" evtable="EVENTS" scfile=september102017,1530-0500_SC00.fits sctable="SC_DATA" outfile=september102017_flare_lt.fits dcostheta=0.025 binsz=1.0 phibins=0 tmin=0.0 tmax=0.0 file_version="1" zmin=0.0 zmax=100.0 chatter=2 clobber=yes debug=no gui=no mode="ql"


Working on file september102017,1530-0500_SC00.fits
........................!
real 0.93
user 0.72
sys 0.25


## **GtExposureMap**

In [10]:
expmap_file = flaredate + '_flare_expmap.fits'
expMap['evfile'] = gti_file
expMap['scfile'] = sc_file
expMap['expcube'] = ltcube_file
expMap['outfile'] = expmap_file
expMap['irfs'] = irfs
expMap['srcrad'] = 20
expMap['nlong'] = dim
expMap['nlat'] = dim
expMap['nenergies'] = 35
expMap.run()

time -p gtexpmap evfile=september102017_flare_gti.fits evtable="EVENTS" scfile=september102017,1530-0500_SC00.fits sctable="SC_DATA" expcube=september102017_flare_lt.fits outfile=september102017_flare_expmap.fits irfs="P8R3_TRANSIENT015S_V2" evtype="INDEF" srcrad=20.0 nlong=100 nlat=100 nenergies=35 submap=no nlongmin=0 nlongmax=0 nlatmin=0 nlatmax=0 chatter=2 clobber=yes debug=no gui=no mode="ql"


The exposure maps generated by this tool are meant
to be used for *unbinned* likelihood analysis only.
Do not use them for binned analyses.
Computing the ExposureMap using september102017_flare_lt.fits
....................!
real 315.95
user 291.56
sys 10.32


## **Generate XML Model**

make sure to go to [here](https://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html) to download the appropriate model from the Fermi Analysis Threads

In [11]:
from make4FGLxml import *
mymodel = srcList('gll_psc_v21.fit',gti_file,flaredate + 'flare_model.xml')
mymodel.makeModel('gll_iem_v07.fits', 'gll_iem_v07', irf_name + '.txt', irf_name)

This is make4FGLxml version 01r05.
The default diffuse model files and names are for pass 8 and 4FGL and assume you have v11r5p3 of the Fermi Science Tools or higher.
Creating file and adding sources from 4FGL
Added 138 point sources and 0 extended sources


## **GtDiffuseResponse**

In [ ]:
diffResps['evfile'] = gti_file
diffResps['scfile'] = sc_file
diffResps['srcmdl'] = flaredate + 'flare_model.xml'
diffResps['irfs'] = irfs
diffResps.run()

time -p gtdiffrsp evfile=september102017_flare_gti.fits evtable="EVENTS" scfile=september102017,1530-0500_SC00.fits sctable="SC_DATA" srcmdl=september102017flare_model.xml irfs="P8R3_TRANSIENT015S_V2" evclsmin=0 evclass="INDEF" evtype="INDEF" convert=no chatter=2 clobber=no debug=no gui=no mode="ql"


## **GtLike**

In [ ]:
import pyLikelihood
from UnbinnedAnalysis import *
obs = UnbinnedObs(gti_file,sc_file,expMap=expmap_file,expCube=ltcube_file,irfs=irfs)
like = UnbinnedAnalysis(obs,flaredate + 'flare_model.xml',optimizer=optimizer)

In [ ]:
obs

In [ ]:
print(like.tol)
print(like.tolType)
like.tol = 0.0001

In [ ]:
likeobj = pyLike.NewMinuit(like.logLike)
like.fit(verbosity=0,covar=True,optObject=likeobj)

In [ ]:
like.plot()

In [ ]:
likeobj.getRetCode()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
E = (like.energies[:-1] + like.energies[1:])/2.
#The 'energies' array are the endpoints so we take the midpoint of the bins.
plt.figure(figsize=(5,5))
plt.ylim((0.4,1e4))
plt.xlim((200,300000))
sum_model = np.zeros_like(like._srcCnts(like.sourceNames()[0]))
for sourceName in like.sourceNames():
    sum_model = sum_model + like._srcCnts(sourceName)
    plt.loglog(E,like._srcCnts(sourceName),label=sourceName[1:])
plt.loglog(E,sum_model,label='Total Model')
plt.errorbar(E,like._Nobs(),yerr=np.sqrt(like._Nobs()), fmt='o',label='Counts')
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.5, box.height])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
resid = (like._Nobs() - sum_model)/sum_model
resid_err = (np.sqrt(like._Nobs())/sum_model)
plt.figure(figsize=(9,9))
plt.xscale('log')
plt.errorbar(E,resid,yerr=resid_err,fmt='o')
plt.axhline(0.0,ls=':')
plt.show()

In [ ]:
like.model['']